In [547]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def gdf(data):
    return pd.read_csv(StringIO(data))

In [440]:
df = pd.DataFrame(  {
'stock': ['AAPL', 'AAPL', 'AAPL','AAPL', 'MSFT', 'MSFT','MSFT', 'MSFT'], 
'datetime': ['2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05'],
'Dummy': [0, 0, 1, 1, 1,1, 0, 1],
})
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index(['stock', 'datetime'], inplace =True)
s = df['Dummy']
m = s.ne(s.shift())
df['Counter'] = s.mask(s.eq(0), -1).groupby(['stock', m.cumsum()]).cumsum()
df

Dummy  Counter
stock datetime                  
AAPL  2015-01-02      0       -1
      2015-01-03      0       -2
      2015-01-04      1        1
      2015-01-05      1        2
MSFT  2015-01-02      1        1
      2015-01-03      1        2
      2015-01-04      0       -1
      2015-01-05      1        1

In [429]:
d="""Date,Hour,Currency,Price,Variation
2021-05-01,23:19:21,BAT,1.0700,NaN
2021-05-01,23:19:21,BTC,47922.1400,NaN
2021-05-01,23:19:21,DOGE,0.3286,NaN
2021-05-01,23:19:21,ETH,2451.7400,NaN
2021-05-01,23:35:50,BAT,1.0600,-0.934579
2021-05-01,23:35:50,BTC,47557.2700,-0.761381
2021-05-01,23:35:50,DOGE,0.3228,-1.765064
2021-05-01,23:35:50,ETH,2438.0300,-0.559195
2021-05-01,23:37:20,BAT,1.0500,-0.943396
2021-05-01,23:37:20,BTC,47467.0200,-0.189771
2021-05-01,23:37:20,DOGE,0.3209,-0.588600
2021-05-01,23:37:20,ETH,2435.3000,-0.111976
2021-05-02,00:04:40,BAT,1.0200,NaN
2021-05-02,00:04:40,BTC,46883.6300,-1.229043
2021-05-02,00:04:40,DOGE,0.3028,NaN
2021-05-02,00:04:40,ETH,2397.8200,NaN"""
df=gdf(d)
df

,Date,Hour,Currency,Price,Variation
0,2021-05-01,23:19:21,BAT,1.0700,NaN
1,2021-05-01,23:19:21,BTC,47922.1400,NaN
2,2021-05-01,23:19:21,DOGE,0.3286,NaN
3,2021-05-01,23:19:21,ETH,2451.7400,NaN
4,2021-05-01,23:35:50,BAT,1.0600,-0.934579
5,2021-05-01,23:35:50,BTC,47557.2700,-0.761381
6,2021-05-01,23:35:50,DOGE,0.3228,-1.765064
7,2021-05-01,23:35:50,ETH,2438.0300,-0.559195
8,2021-05-01,23:37:20,BAT,1.0500,-0.943396
9,2021-05-01,23:37:20,BTC,47467.0200,-0.189771


In [412]:
df['Variation'] = 100*df.groupby('Currency').Price.pct_change()

In [411]:
df

,Date,Hour,Currency,Price,Variation
0,2021-05-01,23:19:21,BAT,1.0700,NaN
1,2021-05-01,23:19:21,BTC,47922.1400,NaN
2,2021-05-01,23:19:21,DOGE,0.3286,NaN
3,2021-05-01,23:19:21,ETH,2451.7400,NaN
4,2021-05-01,23:35:50,BAT,1.0600,-0.009346
5,2021-05-01,23:35:50,BTC,47557.2700,-0.007614
6,2021-05-01,23:35:50,DOGE,0.3228,-0.017651
7,2021-05-01,23:35:50,ETH,2438.0300,-0.005592
8,2021-05-01,23:37:20,BAT,1.0500,-0.009434
9,2021-05-01,23:37:20,BTC,47467.0200,-0.001898


In [430]:
df['Variation'] = df.groupby('Currency').Price.transform(lambda x: 100*x.diff()/x)
df

,Date,Hour,Currency,Price,Variation
0,2021-05-01,23:19:21,BAT,1.0700,NaN
1,2021-05-01,23:19:21,BTC,47922.1400,NaN
2,2021-05-01,23:19:21,DOGE,0.3286,NaN
3,2021-05-01,23:19:21,ETH,2451.7400,NaN
4,2021-05-01,23:35:50,BAT,1.0600,-0.943396
5,2021-05-01,23:35:50,BTC,47557.2700,-0.767222
6,2021-05-01,23:35:50,DOGE,0.3228,-1.796778
7,2021-05-01,23:35:50,ETH,2438.0300,-0.562339
8,2021-05-01,23:37:20,BAT,1.0500,-0.952381
9,2021-05-01,23:37:20,BTC,47467.0200,-0.190132


In [442]:
d="""tweets,datetime
'Everybody...',datetime.datetime(2020, 8, 29, 23, 25, 19)
'Good for ...',datetime.datetime(2020, 8, 29, 22, 31, 26)
'tweeting ...',datetime.datetime(2020, 8, 29, 21, 1, 21)
'rockets on...',datetime.datetime(2020, 8, 29, 20, 23, 57)"""
df=gdf(d)
df

,,,,,tweets,datetime
'Everybody...',datetime.datetime(2020,8,29,23,25,19)
'Good for ...',datetime.datetime(2020,8,29,22,31,26)
'tweeting ...',datetime.datetime(2020,8,29,21,1,21)
'rockets on...',datetime.datetime(2020,8,29,20,23,57)


In [466]:
d="""payeeId,amount,createdAt,TrxnID
1001,2.30,2021-04-24 01:40:11.156000+00:00,100AA
1001,35,2021-04-24 02:10:11.146000+00:00,100AB
1001,600,2021-04-24 02:12:14.309000+00:00,100AC
1002,100,2021-04-24 02:59:51.127000+00:00,110BD
1003,1900,2021-04-24 04:09:15.113000+00:00,120AC
1003,10,2021-04-24 04:19:40.132000+00:00,120AM"""
df=gdf(d)
df['createdAt'] = pd.to_datetime(df.createdAt)
#df = df.set_index('createdAt')
df

,payeeId,amount,createdAt,TrxnID
0,1001,2.3,2021-04-24 01:40:11.156000+00:00,100AA
1,1001,35.0,2021-04-24 02:10:11.146000+00:00,100AB
2,1001,600.0,2021-04-24 02:12:14.309000+00:00,100AC
3,1002,100.0,2021-04-24 02:59:51.127000+00:00,110BD
4,1003,1900.0,2021-04-24 04:09:15.113000+00:00,120AC
5,1003,10.0,2021-04-24 04:19:40.132000+00:00,120AM


In [468]:
diff_sec=300.
mask= df.groupby('payeeId').createdAt.diff().astype('timedelta64[s]').fillna(diff_sec)
# plz replace 'Not Approved' with 'No Settlement' based on requirement.
df['Flag'] = np.where(mask >= diff_sec, 'Approved', 'Not Approved')
df

,payeeId,amount,createdAt,TrxnID,Flag
0,1001,2.3,2021-04-24 01:40:11.156000+00:00,100AA,Approved
1,1001,35.0,2021-04-24 02:10:11.146000+00:00,100AB,Approved
2,1001,600.0,2021-04-24 02:12:14.309000+00:00,100AC,Not Approved
3,1002,100.0,2021-04-24 02:59:51.127000+00:00,110BD,Approved
4,1003,1900.0,2021-04-24 04:09:15.113000+00:00,120AC,Approved
5,1003,10.0,2021-04-24 04:19:40.132000+00:00,120AM,Approved


In [477]:
df = pd.DataFrame({'account_id': {0: 1, 1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 3},
 'contract_id': {0: 'A', 1: 'B', 2: 'C', 3: 'K', 4: 'F', 5: 'Z', 6: 'Y'},
 'date_activated': {0: Timestamp('2020-12-20 00:00:00'),
  1: Timestamp('2020-12-25 00:00:00'),
  2: Timestamp('2021-03-06 00:00:00'),
  3: Timestamp('2020-12-01 00:00:00'),
  4: Timestamp('2021-02-23 00:00:00'),
  5: Timestamp('2020-12-01 00:00:00'),
  6: Timestamp('2021-01-20 00:00:00')},
 'term_months': {0: 1, 1: 12, 2: 12, 3: 4, 4: 6, 5: 12, 6: 12},
 'contract_type': {0: 'Original',
  1: 'Upgrade',
  2: 'Upgrade',
  3: 'Original',
  4: 'Renewal',
  5: 'Original',
  6: 'Upgrade'},
 'renewal_type': {0: 'Regular',
  1: 'Regular',
  2: 'Early',
  3: 'Regular',
  4: 'Early',
  5: 'Regular',
  6: 'Early'},
 datetime.datetime(2020, 12, 1, 0, 0): {0: 200.0,
  1: 300.0,
  2: 0.0,
  3: 100.0,
  4: 0.0,
  5: 80.0,
  6: 0.0},
 datetime.datetime(2021, 1, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 100.0,
  4: 0.0,
  5: 0.0,
  6: 90.0},
 datetime.datetime(2021, 2, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 100.0,
  4: 50.0,
  5: 0.0,
  6: 0.0},
 datetime.datetime(2021, 3, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 400.0,
  3: 0.0,
  4: 50.0,
  5: 0.0,
  6: 0.0},
 datetime.datetime(2021, 4, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 400.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0},
 datetime.datetime(2021, 5, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0},
 datetime.datetime(2021, 6, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0}})
df

,account_id,contract_id,date_activated,term_months,contract_type,renewal_type,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00
0,1,A,2020-12-20,1,Original,Regular,200.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,B,2020-12-25,12,Upgrade,Regular,300.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,C,2021-03-06,12,Upgrade,Early,0.0,0.0,0.0,400.0,400.0,0.0,0.0
3,2,K,2020-12-01,4,Original,Regular,100.0,100.0,100.0,0.0,0.0,0.0,0.0
4,2,F,2021-02-23,6,Renewal,Early,0.0,0.0,50.0,50.0,0.0,0.0,0.0
5,3,Z,2020-12-01,12,Original,Regular,80.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3,Y,2021-01-20,12,Upgrade,Early,0.0,90.0,0.0,0.0,0.0,0.0,0.0


In [478]:
dfo = pd.DataFrame({'account_id': {0: 1, 1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 3},
 'contract_id': {0: 'A', 1: 'B', 2: 'C', 3: 'K', 4: 'F', 5: 'Z', 6: 'Y'},
 'date_activated': {0: Timestamp('2020-12-20 00:00:00'),
  1: Timestamp('2020-12-25 00:00:00'),
  2: Timestamp('2021-03-06 00:00:00'),
  3: Timestamp('2020-12-01 00:00:00'),
  4: Timestamp('2021-02-23 00:00:00'),
  5: Timestamp('2020-12-01 00:00:00'),
  6: Timestamp('2021-01-20 00:00:00')},
 'term_months': {0: 1, 1: 12, 2: 12, 3: 4, 4: 6, 5: 12, 6: 12},
 'contract_type': {0: 'Original',
  1: 'Upgrade',
  2: 'Upgrade',
  3: 'Original',
  4: 'Renewal',
  5: 'Original',
  6: 'Upgrade'},
 'renewal_type': {0: 'Regular',
  1: 'Regular',
  2: 'Early',
  3: 'Regular',
  4: 'Early',
  5: 'Regular',
  6: 'Early'},
 'Status': {0: 'Inactive',
  1: 'Active',
  2: 'Active',
  3: 'Inactive',
  4: 'Active',
  5: 'Inactive',
  6: 'Inactive'},
 datetime.datetime(2020, 12, 1, 0, 0): {0: 200.0,
  1: 300.0,
  2: 0.0,
  3: 100.0,
  4: 0.0,
  5: 80.0,
  6: 0.0},
 datetime.datetime(2021, 1, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 100.0,
  4: 0.0,
  5: 0.0,
  6: 90.0},
 datetime.datetime(2021, 2, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 100.0,
  4: 50.0,
  5: 0.0,
  6: 0.0},
 datetime.datetime(2021, 3, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 400.0,
  3: 0.0,
  4: 50.0,
  5: 0.0,
  6: 0.0},
 datetime.datetime(2021, 4, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 400.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0},
 datetime.datetime(2021, 5, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0},
 datetime.datetime(2021, 6, 1, 0, 0): {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0}})
dfo

,account_id,contract_id,date_activated,term_months,contract_type,renewal_type,Status,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00
0,1,A,2020-12-20,1,Original,Regular,Inactive,200.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,B,2020-12-25,12,Upgrade,Regular,Active,300.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,C,2021-03-06,12,Upgrade,Early,Active,0.0,0.0,0.0,400.0,400.0,0.0,0.0
3,2,K,2020-12-01,4,Original,Regular,Inactive,100.0,100.0,100.0,0.0,0.0,0.0,0.0
4,2,F,2021-02-23,6,Renewal,Early,Active,0.0,0.0,50.0,50.0,0.0,0.0,0.0
5,3,Z,2020-12-01,12,Original,Regular,Inactive,80.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3,Y,2021-01-20,12,Upgrade,Early,Inactive,0.0,90.0,0.0,0.0,0.0,0.0,0.0


In [528]:
data= pd.DataFrame({
    'last_visit_time':['16:08:02', '04:14:05']
}, dtype='string')
#data['last_visit_time'] = pd.to_datetime(data.last_visit_time, format='%H:%M:%S')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   last_visit_time  2 non-null      string
dtypes: string(1)
memory usage: 144.0 bytes


In [521]:
pd.to_timedelta(data.last_visit_time)

0   0 days 16:08:02
1   0 days 04:14:05
Name: last_visit_time, dtype: timedelta64[ns]

In [524]:
data['last_visit_time'] = pd.to_timedelta(data['last_visit_time'])
data

,last_visit_time
0,0 days 16:08:02
1,0 days 04:14:05


In [529]:
pd.to_datetime(data.last_visit_time).dt.time

0    16:08:02
1    04:14:05
Name: last_visit_time, dtype: object

In [531]:
pd.DatetimeIndex(data.last_visit_time)

DatetimeIndex(['2021-05-02 16:08:02', '2021-05-02 04:14:05'], dtype='datetime64[ns]', name='last_visit_time', freq=None)

In [540]:
d="""COURSE_KEY,CORP_ID
2.97E+11,23096
2.97E+11,23097
2.97E+11,10987
2.97E+11,560989
2.97E+11,34678"""
df1=gdf(d)
df1

,COURSE_KEY,CORP_ID
0,2.970000e+11,23096
1,2.970000e+11,23097
2,2.970000e+11,10987
3,2.970000e+11,560989
4,2.970000e+11,34678


In [541]:
d = """COURSE_KEY,COURSE_UNIQUE_KEY,COURSE_ID,CERTIFICATION_ID
2.97E+11,4077,WW_13456,WFT-CK-027
2.97E+11,5789,ww_13456,NL-WFT-12121"""
df2 = gdf(d)

df2

,COURSE_KEY,COURSE_UNIQUE_KEY,COURSE_ID,CERTIFICATION_ID
0,2.970000e+11,4077,WW_13456,WFT-CK-027
1,2.970000e+11,5789,ww_13456,NL-WFT-12121


KeyError: "None of [Float64Index([297000000000.0, 297000000000.0, 297000000000.0, 297000000000.0,\n              297000000000.0],\n             dtype='float64')] are in the [columns]"

In [551]:
d="""datetime,A
2012-08-14 07:00,1
2012-08-14 07:30,300
2012-08-14 08:15,3
2013-08-14 16:00,600
2013-08-14 16:15,700"""
df1=gdf(d)
df1

,datetime,A
0,2012-08-14 07:00,1
1,2012-08-14 07:30,300
2,2012-08-14 08:15,3
3,2013-08-14 16:00,600
4,2013-08-14 16:15,700


In [552]:
d="""datetime,B
2012-08-14 07:00,100
2012-08-14 07:15,200
2012-08-14 07:30,300
2013-09-24 15:45, 50
2013-09-24 16:00, 60
2013-09-24 16:15, 70"""
df2=gdf(d)
df2

,datetime,B
0,2012-08-14 07:00,100
1,2012-08-14 07:15,200
2,2012-08-14 07:30,300
3,2013-09-24 15:45,50
4,2013-09-24 16:00,60
5,2013-09-24 16:15,70


In [553]:
df1.merge(df2, left_on=['datetime', 'A'], right_on=['datetime', 'B'],
         how='inner')

,datetime,A,B
0,2012-08-14 07:30,300,300


In [582]:
df = pd.DataFrame({
    'Date':[16234.0, 16003.]
})
#epoch = pd.to_datetime(0, unit='s').to_julian_date()
#pd.to_datetime(epoch - df.Date, unit='D')
pd.to_datetime('2016.01.03').to_julian_date()

pd.to_datetime('2016.01.03') - pd.DateOffset(days=16234)

Timestamp('1971-07-24 00:00:00')